In [1]:
!nvidia-smi

Sat Mar 18 13:22:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
import numpy as np


In [4]:
df = pd.read_csv('./train.csv')
df.head()

,ID,가입일,음성사서함이용,주간통화시간,주간통화횟수,주간통화요금,저녁통화시간,저녁통화횟수,저녁통화요금,밤통화시간,밤통화횟수,밤통화요금,상담전화건수,전화해지여부
0,TRAIN_00000,329,0,99.2,93,27.3,268.8,68,28.92,262.9,328,32.89,2,0
1,TRAIN_00001,2,80,323.9,323,83.7,269.4,326,32.09,322.8,209,32.32,2,0
2,TRAIN_00002,93,28,282.4,323,34.2,207.0,322,32.82,280.8,328,8.28,0,0
3,TRAIN_00003,223,1,221.4,223,25.1,233.0,61,23.90,203.8,234,9.36,0,0
4,TRAIN_00004,222,0,96.3,222,28.7,223.9,69,28.08,263.1,223,2.80,8,0


In [5]:
df.shape

(30200, 14)

In [6]:
df['전화해지여부'].value_counts()

0    26882
1     3318
Name: 전화해지여부, dtype: int64

In [7]:
df['가입일'].describe()

count    30200.000000
mean       159.732649
std        123.752781
min          1.000000
25%         63.000000
50%        118.000000
75%        228.000000
max       2212.000000
Name: 가입일, dtype: float64

In [9]:
df['음성사서함이용'].describe()

count    30200.000000
mean        12.037815
std         24.925334
min          0.000000
25%          0.000000
50%          1.000000
75%         22.000000
max       1112.000000
Name: 음성사서함이용, dtype: float64

In [10]:
df['음성사서함이용'].value_counts()

0       10988
1       10852
23        825
22        669
29        610
        ...  
117         1
13          1
116         1
111         1
1112        1
Name: 음성사서함이용, Length: 76, dtype: int64

In [11]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [12]:
df.drop(['ID'],axis=1, inplace=True)
df.head()

,가입일,음성사서함이용,주간통화시간,주간통화횟수,주간통화요금,저녁통화시간,저녁통화횟수,저녁통화요금,밤통화시간,밤통화횟수,밤통화요금,상담전화건수,전화해지여부
0,329,0,99.2,93,27.3,268.8,68,28.92,262.9,328,32.89,2,0
1,2,80,323.9,323,83.7,269.4,326,32.09,322.8,209,32.32,2,0
2,93,28,282.4,323,34.2,207.0,322,32.82,280.8,328,8.28,0,0
3,223,1,221.4,223,25.1,233.0,61,23.90,203.8,234,9.36,0,0
4,222,0,96.3,222,28.7,223.9,69,28.08,263.1,223,2.80,8,0


In [13]:
def func_0(time):
    if time <= 63:
        return 1
    elif time <=118:
        return 2
    elif time <=228:
        return 3
    else:
        return 4

In [14]:
df['가입일분위수'] = df['가입일'].apply(lambda x: func_0(x))

In [15]:
def func(cnt):
    if cnt == 0:
        return 1
    elif cnt ==1:
        return 2
    else:
        return 3

In [16]:
df['음성사서함이용분위수'] = df['음성사서함이용'].apply(lambda x: func(x))

In [17]:
def func_1(time):
    if time <= 201: # 하위 12프로
        return 1
    elif time <=234:
        return 2
    elif time <=289:
        return 3
    else:
        return 4

In [18]:
df['주간통화시간분위수'] = df['주간통화시간'].apply(lambda x: func_1(x))

In [19]:
def func_2(cnt):
    if cnt <= 88:
        return 1
    elif cnt <=117:
        return 2
    elif cnt <=223:
        return 3
    else:
        return 4

In [20]:
df['주간통화횟수분위수'] = df['주간통화횟수'].apply(lambda x: func_2(x))

In [21]:
def func_3(fee):
    if fee <= 24.4:
        return 1
    elif fee <=30.8:
        return 2
    elif fee <=45.9:
        return 3
    else:
        return 4

In [22]:
df['주간통화요금분위수'] = df['주간통화요금'].apply(lambda x: func_3(x))

In [23]:
df['저녁통화시간분위수'] = df['저녁통화시간'].apply(lambda x: func_1(x))

In [24]:
df['저녁통화횟수분위수'] = df['저녁통화횟수'].apply(lambda x: func_2(x))

In [25]:
def func_4(fee):
    if fee <= 22.2:
        return 1
    elif fee <=23.32:
        return 2
    elif fee <=28.82:
        return 3
    else:
        return 4

In [26]:
df['저녁통화요금분위수'] = df['저녁통화요금'].apply(lambda x: func_4(x))

In [27]:
df['밤통화시간분위수'] = df['밤통화시간'].apply(lambda x: func_1(x))

In [28]:
df['밤통화횟수분위수'] = df['밤통화횟수'].apply(lambda x: func_2(x))

In [29]:
def func_5(fee):
    if fee <= 4.7:
        return 1
    elif fee <=9.0:
        return 2
    elif fee <=20.29:
        return 3
    else:
        return 4

In [30]:
df['밤통화요금분위수'] = df['밤통화요금'].apply(lambda x: func_5(x))

In [31]:
df.drop(columns=['가입일','음성사서함이용','주간통화시간','주간통화횟수','주간통화요금','저녁통화시간','저녁통화횟수','저녁통화요금', '밤통화시간', '밤통화횟수','밤통화요금'], axis=1, inplace=True)

In [32]:
y = df['전화해지여부']
y.head()

0    0
1    0
2    0
3    0
4    0
Name: 전화해지여부, dtype: int64

In [33]:
y.value_counts()

0    26882
1     3318
Name: 전화해지여부, dtype: int64

In [34]:
x = df.drop(['전화해지여부'],axis=1)
x.head()

,상담전화건수,가입일분위수,음성사서함이용분위수,주간통화시간분위수,주간통화횟수분위수,주간통화요금분위수,저녁통화시간분위수,저녁통화횟수분위수,저녁통화요금분위수,밤통화시간분위수,밤통화횟수분위수,밤통화요금분위수
0,2,4,1,1,2,2,3,1,4,3,4,4
1,2,1,3,4,4,4,3,4,4,4,3,4
2,0,2,3,3,4,3,2,4,4,3,4,2
3,0,3,2,2,3,2,2,1,3,2,4,3
4,8,3,1,1,3,2,2,1,3,3,3,1


In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
lr_clf = LogisticRegression()
dt_clf = DecisionTreeClassifier(random_state=42)
rf_clf = RandomForestClassifier(random_state=42, n_estimators=1000)
# ada_clf = AdaBoostClassifier(n_estimators=1000)
# gb_clf = GradientBoostingClassifier(random_state=42, n_estimators=1000)
xgb_clf = XGBClassifier(n_estimators=1000)
lgb_clf = LGBMClassifier(n_estimators=1000,boost_from_average=False)
vo_clf = VotingClassifier(estimators=[('DT',dt_clf),('RF',rf_clf), ('XGB',xgb_clf),('LGB',lgb_clf)], voting='soft')

models = [dt_clf, rf_clf, xgb_clf, lgb_clf,vo_clf]
# # models = [dt_reg, rf_reg, gb_reg]
# for model in models:
#     model.fit(x_train,y_train)
#     score = model.score(x_test,y_test)
#     print('{0}의 점수:{1:.2f}'.format(model,score))

In [36]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)

In [37]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)
best_score = 0
best_model = "None" 
for i, (train_index, val_index) in enumerate (skf.split(x,y)):
    print(f'Fold{i} start!')
    train_cv = x.iloc[train_index]
    y_cv = y.iloc[train_index]
    valid_cv = x.iloc[val_index]
    yvalid_cv = y.iloc[val_index]
    x_train=train_cv.loc[:,:].reset_index(drop=True)
    y_train=y_cv.loc[:].reset_index(drop=True)

    x_train_over, y_train_over = smote.fit_resample(x_train, y_train)
    print("SMOTE 적용 전 학습용 피처/레이블 데이터 세트:", x_train.shape, y_train.shape)
    print("SMOTE 적용 후 학습용 피처/레이블 데이터 세트:", x_train_over.shape, y_train_over.shape)
    print("SMOTE 적용 후 레이블 값 분포: \n", pd.Series(y_train_over).value_counts())
    x_val = valid_cv.loc[:,:].reset_index(drop=True)
    y_val=yvalid_cv.loc[:].reset_index(drop=True)
    sub_pred = []
    for model in models:
        model.fit(x_train_over,y_train_over)
        model_pred = model.predict(x_val)
        sub_pred.append(model_pred)
        val_score = model.score(x_val,y_val)
        val_accuracy = accuracy_score(y_val, model_pred)
      # print('{0}의 점수:{1:.4f}'.format(model,val_score))
        print('{0}의 정확도:{1:.4f}'.format(model, val_accuracy))
        if best_score < val_accuracy:
            best_model = model
            best_score = val_accuracy
    # pred = np.array(sub_pred)
    # print(pred.shape)
    # pred = np.transpose(pred)
    # print(pred.shape)
    # lr_clf.fit(pred,y_val)
    # final = lr_clf.predict(pred)
    # print("stacking 앙상블 예측 정확도: {0:.4f}".format(accuracy_score(y_val,final)))
    # final_accuracy = accuracy_score(y_val,final) 
    # if best_score < final_accuracy:
    #     best_model = lr_clf
    #     best_score = final_accuracy

Fold0 start!
SMOTE 적용 전 학습용 피처/레이블 데이터 세트: (27180, 12) (27180,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트: (48386, 12) (48386,)
SMOTE 적용 후 레이블 값 분포: 
 0    24193
1    24193
Name: 전화해지여부, dtype: int64
DecisionTreeClassifier(random_state=42)의 정확도:0.8560
RandomForestClassifier(n_estimators=1000, random_state=42)의 정확도:0.9020
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=None, num_parallel_tr

In [38]:
test = pd.read_csv('./test.csv')
test.head()

,ID,가입일,음성사서함이용,주간통화시간,주간통화횟수,주간통화요금,저녁통화시간,저녁통화횟수,저녁통화요금,밤통화시간,밤통화횟수,밤통화요금,상담전화건수
0,TEST_00000,110,0,210.7,142,35.6,205.9,93,17.43,120.9,111,5.37,2
1,TEST_00001,283,1,268.5,100,30.3,283.0,92,28.32,290.0,89,8.82,0
2,TEST_00002,34,1,243.3,205,21.3,225.7,205,29.00,92.6,29,2.44,2
3,TEST_00003,209,86,83.0,93,24.1,270.6,220,22.89,266.9,81,2.80,6
4,TEST_00004,202,0,293.2,23,31.0,294.0,24,26.39,223.8,33,6.66,2


In [39]:
test['가입일분위수'] = test['가입일'].apply(lambda x: func_0(x))

In [40]:
test['음성사서함이용분위수'] = test['음성사서함이용'].apply(lambda x: func(x))

In [41]:
test['주간통화시간분위수'] = test['주간통화시간'].apply(lambda x: func_1(x))

In [42]:
test['주간통화횟수분위수'] = test['주간통화횟수'].apply(lambda x: func_2(x))

In [43]:
test['주간통화요금분위수'] = test['주간통화요금'].apply(lambda x: func_3(x))

In [44]:
test['저녁통화시간분위수'] = test['저녁통화시간'].apply(lambda x: func_1(x))

In [45]:
test['저녁통화횟수분위수'] = test['저녁통화횟수'].apply(lambda x: func_2(x))

In [46]:
test['저녁통화요금분위수'] = test['저녁통화요금'].apply(lambda x: func_4(x))

In [47]:
test['밤통화시간분위수'] = test['밤통화시간'].apply(lambda x: func_1(x))

In [48]:
test['밤통화횟수분위수'] = test['밤통화횟수'].apply(lambda x: func_2(x))

In [49]:
test['밤통화요금분위수'] = test['밤통화요금'].apply(lambda x: func_5(x))

In [50]:
test.drop(columns=['주간통화시간','주간통화횟수','주간통화요금','저녁통화시간','저녁통화횟수','저녁통화요금', '밤통화시간', '밤통화횟수','밤통화요금'], axis=1, inplace=True)

In [51]:
test.head()

,ID,가입일,음성사서함이용,상담전화건수,가입일분위수,음성사서함이용분위수,주간통화시간분위수,주간통화횟수분위수,주간통화요금분위수,저녁통화시간분위수,저녁통화횟수분위수,저녁통화요금분위수,밤통화시간분위수,밤통화횟수분위수,밤통화요금분위수
0,TEST_00000,110,0,2,2,1,2,3,3,2,2,1,1,2,2
1,TEST_00001,283,1,0,4,2,3,2,2,3,2,3,4,2,2
2,TEST_00002,34,1,2,1,2,3,3,1,2,3,4,1,1,1
3,TEST_00003,209,86,6,3,3,1,2,1,3,3,2,3,1,1
4,TEST_00004,202,0,2,3,1,4,1,3,4,1,3,2,1,2


In [52]:
test.drop(['ID','가입일','음성사서함이용'],axis=1, inplace=True)

In [53]:
x_t = test.values
x_t

array([[2, 2, 1, ..., 1, 2, 2],
       [0, 4, 2, ..., 4, 2, 2],
       [2, 1, 2, ..., 1, 1, 1],
       ...,
       [2, 2, 3, ..., 2, 3, 4],
       [0, 1, 3, ..., 2, 1, 3],
       [2, 1, 3, ..., 2, 1, 1]])

In [54]:
x_t.shape

(12943, 12)

In [55]:
if best_model == lr_clf:
    sub_pred = []
    for model in models:
        model_pred = model.predict(x_t)
        sub_pred.append(model_pred)
    x_pred = np.array(sub_pred)
    print(x_pred.shape)
    x_pred = np.transpose(x_pred)
    print(x_pred.shape)

In [57]:
y_r_pred = best_model.predict(x_t)
dr = pd.DataFrame(y_r_pred)
dr

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


,0
0,0
1,0
2,0
3,1
4,1
...,...
12938,0
12939,0
12940,0
12941,0


In [58]:
submission = pd.read_csv('./sample_submission.csv')

In [59]:
submission['전화해지여부'] = dr
submission

,ID,전화해지여부
0,TEST_00000,0
1,TEST_00001,0
2,TEST_00002,0
3,TEST_00003,1
4,TEST_00004,1
...,...,...
12938,TEST_12938,0
12939,TEST_12939,0
12940,TEST_12940,0
12941,TEST_12941,0


In [60]:
submission.to_csv('Kfold_SMOTE_all_use.csv', index=False)